In [2]:
import os
from google.cloud import bigquery
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import math
import re
from newspaper import Article
import nltk
from nltk.stem import PorterStemmer
import json 

In [3]:
# # Specify credentials
# credential_path=".config/gcloud/application_default_credentials.json"
# os.environ['GOOGLE_APPLICATION_CREDENTIALS']=credential_path
# bigquery_client = bigquery.Client(project='ttds2-338418')

In [4]:
# QUERY="SELECT DocumentIdentifier, TranslationInfo, Extras, SourceCollectionIdentifier, DATE, V2Themes, V2Organizations, AllNames FROM `gdelt-bq.gdeltv2.gkg` where DATE>20220119000000 and DATE<20220127000000 and V2Themes like '%ECON_STOCKMARKET%' or V2Themes like '%ECON_EARNINGS_REPORT%' or V2Themes like '%ECON_IPO%'"

# query_job=bigquery_client.query(QUERY) #API request

# df = query_job.to_dataframe()

In [ ]:
# Remove duplicate links
df=df.drop_duplicates(subset=['DocumentIdentifier'])

In [73]:
#df.to_csv('gd_ouput.csv')
df=pd.read_csv("data_backup/gd_ouput.csv", nrows=30000)
df = df.drop('Unnamed: 0', 1)

/var/folders/z7/nqjyxg1n1p9073g7jsxhq8w80000gn/T/ipykernel_1216/4252170076.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df.drop('Unnamed: 0', 1)


In [74]:
df.head()

,DocumentIdentifier,TranslationInfo,Extras,SourceCollectionIdentifier,DATE,V2Themes,V2Organizations,AllNames
0,https://www.aleqt.com/2021/09/18/article_21735...,srclc:ara;eng:GT-ARA 1.0,<PAGE_PRECISEPUBTIMESTAMP>20210918165500</PAGE...,1,20210918171500,"HEALTH_VACCINATION,416;UNGP_HEALTHCARE,416;ECO...","York Stock Exchange,6375;New York Stock Exchan...","Saudi Arabia,37;Saudi Arabia,125;Standard Land..."
1,https://www.geekwire.com/2021/proved-wrong-inv...,NaN,<PAGE_LINKS>https://ringingthebell.substack.co...,1,20210918181500,"TAX_FNCACT_HERO,1133;MEDIA_MSM,1627;ECON_IPO,9...","Western Union,249","Remitly Photo,17;Western Union,264;Matt Oppenh..."
2,https://finance.sina.com.cn/tech/2021-09-18/do...,srclc:zho;eng:GT-ZHO 1.0,<PAGE_AUTHORS>@@=value.author$$</PAGE_AUTHORS>...,1,20210918154500,"WB_696_PUBLIC_SECTOR_MANAGEMENT,3274;WB_2048_C...","Step The Association,3724;Management The Rules...","Management China,822;Management Guidelines,126..."
3,https://zdnet.co.kr/view/?no=20210919001641,srclc:kor;eng:GT-KOR 1.0,<PAGE_LINKS>https://zdnet.co.kr/view/?no=20210...,1,20210919010000,"WB_566_ENVIRONMENT_AND_NATURAL_RESOURCES,1275;...","Korea Advanced Institute Of Science,2000","United States Professional Business Section,88..."
4,https://kvia.com/news/2021/11/01/tiktok-rival-...,NaN,<PAGE_LINKS>https://edition.cnn.com/2021/03/18...,1,20211101073000,"GENERAL_GOVERNMENT,1433;EPU_POLICY_GOVERNMENT,...","Cnn,21","Michelle Toh,16;Cheng Yixiao,406;Zhang Yiming,..."


# Part 1 - Pre-processing

In [5]:
def rm_non_english(df):
    # 1) Remove articles not in english
    indexes_to_drop = []

    for index, row in df.iterrows():
        language= row['TranslationInfo']
        # Articles in english have a translationInfo of 'NaN', held as a float
        if type(language) != float:
            indexes_to_drop.append(index)

    df.drop(df.index[indexes_to_drop], inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    return df

In [6]:
# 2) Remove articles not from web (not source type 1)
def rm_non_web(df):
    
    indexes_to_drop = []

    for index, row in df.iterrows():
        source_type= row['SourceCollectionIdentifier']
        if source_type != 1:
            indexes_to_drop.append(index)

    df.drop(df.index[indexes_to_drop], inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    return df

In [7]:
# 3) For organizations collate into a {name:count} dictionary
def collate_organizations(df):
    
    indexes_to_drop = []
    for index, row in df.iterrows():
        # Organizations
        organizations= row['V2Organizations']

        if type(organizations)==str:
            organizations=organizations.split(';')
            org_list=[]
            for org in organizations:
                org=org.split(',')[0]
                org_list.append(org)

            org_list={x:org_list.count(x) for x in org_list}

            # Replace item with dictionary
            df.at[index, 'V2Organizations'] = org_list

        if type(organizations)==float:
            indexes_to_drop.append(index)

    df.drop(df.index[indexes_to_drop], inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    return df

In [8]:
# 4) For themes collate into a {name:count} dictionary
def collate_themes(df):
    
    for index, row in df.iterrows():
        # Themes
        themes= row['V2Themes']

        if type(themes)==str:
            themes=themes.split(';')
            themes_list=[]
            for theme in themes:
                theme=theme.split(',')[0]
                themes_list.append(theme)

            themes_list={x:themes_list.count(x) for x in themes_list}

            # Replace item with dictionary
            df.at[index, 'V2Themes'] = themes_list

    df.reset_index(drop=True, inplace=True)
    
    return df

In [9]:
# 5) For Names collate into a {name:count} dictionary
def collate_all_names(df):
    
    for index, row in df.iterrows():
        # Names
        names= row['AllNames']
        if type(names)==str:
            names=names.split(';')
            names_list=[]
            for name in names:
                name=name.split(',')[0]
                names_list.append(name)

            names_list={x:names_list.count(x) for x in names_list}

            # Replace item with dictionary
            df.at[index, 'AllNames'] = names_list

    df = df.reset_index(drop=True)
    
    return df

In [10]:
#5) Extract titles and add to new column 
def extract_titles(df):
    # Add a title column
    
    df["Title"] = 'NaN'

    # Add titles to that columns
    for index, row in df.iterrows():

        title=float("NaN")
        extras=row['Extras']

        if type(extras) == str:

            if '<PAGE_TITLE>' and '</PAGE_TITLE>' in extras:
                # simple example
                pattern = r"<PAGE_TITLE>(.*?)</PAGE_TITLE>"
                title=re.findall(pattern, extras, flags=0)[0]

            # Add to title column
            df.at[index, 'Title'] = title

    df = df.reset_index(drop=True)
    
    return df

In [11]:
def column_organisation(df):
    
    # Remove unecessary columns - TranslationInfo SourceCollectionIdentifier 
    columns = ['TranslationInfo', 'SourceCollectionIdentifier']
    df.drop(columns, inplace=True, axis=1)    
    df["Content"] = 'NaN'

    # Reoganize columns
    df = df.reset_index(drop=True)
    df = df.reindex(columns=['Title','DocumentIdentifier','Content', 'DATE', 'V2Themes','V2Organizations', 'AllNames', 'Extras'])
    
    return df

# Part 1.2 Running Pre-processing scripts

In [82]:
df=rm_non_english(df)
df=rm_non_web(df)
df=collate_organizations(df)
df=collate_themes(df)
df=collate_all_names(df)
df=extract_titles(df)
df=column_organisation(df)

In [83]:
len(df)

12473

In [84]:
df.head()

,Title,DocumentIdentifier,Content,DATE,V2Themes,V2Organizations,AllNames,Extras
0,'They proved me wrong.' Investor who passed on...,https://www.geekwire.com/2021/proved-wrong-inv...,NaN,20210918181500,"{'TAX_FNCACT_HERO': 1, 'MEDIA_MSM': 1, 'ECON_I...",{'Western Union': 1},"{'Remitly Photo': 1, 'Western Union': 1, 'Matt...",<PAGE_LINKS>https://ringingthebell.substack.co...
1,TikTok rival Kuaishou becomes the latest Chine...,https://kvia.com/news/2021/11/01/tiktok-rival-...,NaN,20211101073000,"{'GENERAL_GOVERNMENT': 1, 'EPU_POLICY_GOVERNME...",{'Cnn': 1},"{'Michelle Toh': 1, 'Cheng Yixiao': 1, 'Zhang ...",<PAGE_LINKS>https://edition.cnn.com/2021/03/18...
2,SoftBank : backed VTEX surges 32% in NYSE debu...,https://www.marketscreener.com/quote/stock/SOF...,NaN,20210721191500,"{'TAX_FNCACT_COORDINATORS': 1, 'WB_2299_PIPELI...","{'Morgan Stanley': 1, 'Mcdonald Corp': 1, 'Son...","{'SoftBank Group-backed': 1, 'Sony Corp': 1, '...",<PAGE_AUTHORS>MarketScreener</PAGE_AUTHORS><PA...
3,"Romania: As many as 15,178 people get the Covi...",http://www.romaniapress.com/news=131541,NaN,20210721183000,"{'TAX_ETHNICITY_ROMANIAN': 3, 'TAX_WORLDLANGUA...","{'Opera Plaza Hotel In Cluj-Napoca': 1, 'Bolya...",{'Sfantu Gheorghe County Emergency Hospital': ...,<PAGE_LINKS>http://www.romaniapress.com/news=1...
4,Human capital firm Paycor pops in Nasdaq debut...,https://www.reuters.com/article/paycor-ipo/upd...,NaN,20210721183000,"{'IDEOLOGY': 1, 'GENERAL_HEALTH': 1, 'HEALTH_V...","{'Thomson Reuters Trust Principles': 1, 'Goldm...","{'United States': 2, 'Apax Partners': 1, 'Ohio...",<PAGE_LINKS>http://thomsonreuters.com/en/about...


# Part 1.3 Give Every Document and ID

Going forward, this will find the last ID from the SQL table and start the indexing from there.

In [118]:
def assign_IDs(df):
    starting_id=0
    
    # Create new column
    df["Id"] = 'NaN'
    
    for i, row in df.iterrows():
        
        df.at[i, "Id"]=int(starting_id)
        
        starting_id+=1
    
    
    df = df.reindex(columns=['Id','Title','DocumentIdentifier','Content', 'DATE', 'V2Themes','V2Organizations', 'AllNames', 'Extras'])

    
    return(df)
        

In [14]:
df=assign_IDs(df)

# Part 2.1 - Running Content Scraping and Organization Mentioning Functions

In [15]:
def extract_content(url):

    try:
        article = Article(url)
        article.download()
        article.parse()

        # Add to title colum
        content= article.text
        
        return (content)

    except Exception as e:

        print("Error at Article:", url)
        return (False)

In [16]:
def add_content(df):
    
    # Add titles to that columns
    for i, row in df.iterrows():
        
        print(i)
        link=row['DocumentIdentifier']
        content = extract_content(link)
        df.at[i, 'Content'] = content
        
    return df

In [98]:
df=df.head(3000)
df=add_content(df)

0
1
2
3
Error at Article: http://www.romaniapress.com/news=131541
4
5
6
7
Error at Article: https://www.wbap.com/news/oops-investors-thought-slack-was-about-to-announce-big-news-it-was-old-news/
8
9
10
Error at Article: https://www.sanantoniopost.com/news/271673580/china-greenlights-three-chinext-ipos
11
12
Error at Article: https://www.saltlakecitysun.com/news/271370027/twin-vee-powercats-co-appoints-new-chief-financial-officer
13
14
15
Error at Article: https://www.sydneysun.com/news/271370652/oyo-files-ipo-to-raise-12-billion
16
17
18
Error at Article: https://www.proactiveinvestors.co.uk/companies/news/219813/frontier-ip-sees-maturity-spread-across-portfolio-companies-219813.html
19
Error at Article: https://www.proactiveinvestors.co.uk/companies/news/961966/ftse-dives-on-worries-over-the-uk-economy-gas-prices-furlough-ending-and-chinas-need-for-energy-961966.html
20
Error at Article: https://www.thedailytimes.com/business/tdcx-poised-to-capture-opportunities-from-the-us-100-billio

Error at Article: https://www.saltlakecitysun.com/news/271709824/liberty-resources-acquisition-corp-announces-pricing-of-100000000-initial-public-offering-and-nasdaq-listing
258
Error at Article: https://www.hawaiitelegraph.com/news/271709890/tg-venture-acquisition-corp-announces-closing-of-115-million-initial-public-offering
259
Error at Article: https://www.hawaiitelegraph.com/news/271709698/bodav2-the-most-innovative-reward-token-on-binance-smart-chain-paying-holders-with-busd-stablecoin
260
Error at Article: https://www.stlouisstar.com/news/271709693/the-spac-conference-europe-to-deliver-an-international-perspective-on-spacs
261
Error at Article: https://www.newjerseytelegraph.com/news/271709871/argo-blockchain-plc-announces-disclosure-of-inside-information
262
Error at Article: https://www.newjerseytelegraph.com/news/271710084/seatech-ventures-corp-announces-international-telecommunications-union-itu-r-membership-approval-of-incubatee-angkasa-x
263
264
Error at Article: https://ww

/Users/humzahmalik/opt/anaconda3/lib/python3.9/site-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname EDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


452
453
Error at Article: https://www.estevanmercury.ca/dialogue-health-makes-tsx-debut-with-ipo-pricing-shares-at-12-each-1.24300968
454
Error at Article: https://www.pittsburghstar.com/news/268310964/jaguar-health-announces-selection-of-investment-bank-and-nominated-advisor-for-proposed-merger-of-the-dragon-spac-and-napo-eu-jaguars-italian-subsidiary
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
Error at Article: https://www.clevelandstar.com/news/271173017/digatrade-provides-september-shareholder-update
479
Error at Article: https://www.tennesseedaily.com/news/271173017/digatrade-provides-september-shareholder-update
480
481
482
483
484
485
486
487
Error at Article: https://www.shanghaisun.com/news/271765785/beijing-stock-exchange-sees-more-investors-in-1st-week-of-trading
488
489
490
Error at Article: https://www.californiatelegraph.com/news/270684880/gladstone-land-acquires-organic-blueberry-orchards-in-oregon--california
491
492
4

/Users/humzahmalik/opt/anaconda3/lib/python3.9/site-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname PST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


...

2303
2304
2305
2306
2307
2308
2309
2310
2311
Error at Article: https://www.taiwansun.com/news/272222943/sensex-nifty-likely-to-remain-volatile-in-the-run-up-to-budget
2312
2313
2314
Error at Article: https://www.chinanationalnews.com/news/272218920/money-flowing-out-of-favorite-pandemic-era-stocks
2315
2316
Error at Article: https://www.brazilsun.com/news/272223749/britain-to-back-suspension-of-russia-from-swift-reports
2317
2318
2319
2320
Error at Article: https://www.northkoreatimes.com/news/272218920/money-flowing-out-of-favorite-pandemic-era-stocks
2321
2322
Error at Article: https://www.morning-times.com/state_news/article_9603d223-64e2-59f1-8954-b5dece012466.html
2323
2324
2325
2326
Error at Article: https://www.nepalnational.com/news/272223749/britain-to-back-suspension-of-russia-from-swift-reports
2327
2328
2329
Error at Article: https://www.philippinetimes.com/news/272217381/indian-stocks-dip-for-4th-straight-day-sensex-tumbles-593-points
2330
Error at Article: https://www.tai

2513
2514
2515
2516
2517
2518
Error at Article: https://www.nashvilleherald.com/news/272219308/silver-elephant-clarifies-its-otcqx-trading-symbol-as-siled-until-february-14th-2022
2519
2520
2521
2522
Error at Article: https://www.trinidadtimes.com/news/272219404/us-stocks-sink-nasdaq-composite-sheds-385-points
2523
2524
Error at Article: https://www.afghanistansun.com/news/272219404/us-stocks-sink-nasdaq-composite-sheds-385-points
2525
2526
Error at Article: https://www.pittsburghstar.com/news/272219308/silver-elephant-clarifies-its-otcqx-trading-symbol-as-siled-until-february-14th-2022
2527
2528
Error at Article: https://www.krem.com/article/news/health/panhandle-health-district-reports-an-increase-in-covid-case-numbers/293-c70a2515-3a2c-46c4-941f-25eca9d2f76a
2529
2530
Error at Article: https://www.newzealandstar.com/news/272219788/us-stocks-in-multi-week-loss-nasdaq-down-most-since-financial-crisis
2531
Error at Article: https://www.bruneinews.net/news/272219788/us-stocks-in-multi-w

/Users/humzahmalik/opt/anaconda3/lib/python3.9/site-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname IST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


2598
2599
2600
2601
Error at Article: https://www.proactiveinvestors.com.au/companies/news/961119/revolver-resources-to--supercharge--exploration-at-its-advanced-copper-projects-961119.html
2602
Error at Article: https://www.oann.com/softbank-backed-oyo-to-file-for-1-billion-ipo-next-week-et/
2603
Error at Article: https://www.saltlakecitysun.com/news/272221540/bitcoin-drops-50-of-its-value
2604
2605
2606
2607
Error at Article: https://www.indiagazette.com/news/272221323/latest-count-of-confirmed-covid-19-cases-worldwide-at-1500-gmt-jan-22
2608
2609
2610
Error at Article: https://www.texasguardian.com/news/272221462/xtb-reviews-whats-about-xtb-leverage
2611
2612
2613
2614
2615
2616
2617
2618
2619
2620
2621
2622
2623
2624
Error at Article: https://www.chinanationalnews.com/news/270856383/ipo-reform-on-chinext-board-boosts-chinas-capital-market
2625
2626
2627
2628
2629
Error at Article: https://www.phoenixherald.com/news/270855146/twin-vee-powercats-reports-second-quarter-2021-operationa

2879
2880
2881
Error at Article: https://www.saltlakecitysun.com/news/272106484/following-successful-year-nepra-foods-issues-letter-from-ceo
2882
Error at Article: https://www.californiatelegraph.com/news/272107986/jupiter-wellness-acquisition-corp-announces-upcoming-automatic-unit-separation
2883
Error at Article: https://www.californiatelegraph.com/news/272106484/following-successful-year-nepra-foods-issues-letter-from-ceo
2884
2885
2886
2887
2888
2889
2890
Error at Article: https://www.nashvilleherald.com/news/272110100/high-quality-idos-are-being-prioritized-by-astronaut-launchpad
2891
Error at Article: https://www.londonmercury.com/news/272110100/high-quality-idos-are-being-prioritized-by-astronaut-launchpad
2892
2893
2894
2895
Error at Article: http://www.plymouthherald.co.uk/tragedy-of-the-plymouth-businessman-who-had-a-train-to-catch/story-30211710-detail/story.html
2896
2897
Error at Article: http://themalaysianreserve.com/new/story/pe-firms-look-out-new-sources-value-asia-pac

In [33]:
# df.to_csv("data_backup/articles_with_content.csv", index=False)

In [76]:
df=pd.read_csv("data_backup/articles_with_content.csv",)

In [77]:
df.head()

,Id,Title,DocumentIdentifier,Content,DATE,V2Themes,V2Organizations,AllNames,Extras
0,0,'They proved me wrong.' Investor who passed on...,https://www.geekwire.com/2021/proved-wrong-inv...,Jason Stoffer was still early in his career at...,20210918181500,"{'TAX_FNCACT_HERO': 1, 'MEDIA_MSM': 1, 'ECON_I...",{'Western Union': 1},"{'Remitly Photo': 1, 'Western Union': 1, 'Matt...",<PAGE_LINKS>https://ringingthebell.substack.co...
1,1,TikTok rival Kuaishou becomes the latest Chine...,https://kvia.com/news/2021/11/01/tiktok-rival-...,"By Michelle Toh, CNN Business\n\nKuaishou, the...",20211101073000,"{'GENERAL_GOVERNMENT': 1, 'EPU_POLICY_GOVERNME...",{'Cnn': 1},"{'Michelle Toh': 1, 'Cheng Yixiao': 1, 'Zhang ...",<PAGE_LINKS>https://edition.cnn.com/2021/03/18...
2,2,SoftBank : backed VTEX surges 32% in NYSE debu...,https://www.marketscreener.com/quote/stock/SOF...,SoftBank Group-backed VTEX's stock opened at $...,20210721191500,"{'TAX_FNCACT_COORDINATORS': 1, 'WB_2299_PIPELI...","{'Morgan Stanley': 1, 'Mcdonald Corp': 1, 'Son...","{'SoftBank Group-backed': 1, 'Sony Corp': 1, '...",<PAGE_AUTHORS>MarketScreener</PAGE_AUTHORS><PA...
3,3,"Romania: As many as 15,178 people get the Covi...",http://www.romaniapress.com/news=131541,False,20210721183000,"{'TAX_ETHNICITY_ROMANIAN': 3, 'TAX_WORLDLANGUA...","{'Opera Plaza Hotel In Cluj-Napoca': 1, 'Bolya...",{'Sfantu Gheorghe County Emergency Hospital': ...,<PAGE_LINKS>http://www.romaniapress.com/news=1...
4,4,Human capital firm Paycor pops in Nasdaq debut...,https://www.reuters.com/article/paycor-ipo/upd...,The Nasdaq logo is displayed at the Nasdaq Mar...,20210721183000,"{'IDEOLOGY': 1, 'GENERAL_HEALTH': 1, 'HEALTH_V...","{'Thomson Reuters Trust Principles': 1, 'Goldm...","{'United States': 2, 'Apax Partners': 1, 'Ohio...",<PAGE_LINKS>http://thomsonreuters.com/en/about...


# Part 2.2 - Removing Articles that dont mention companies

In [78]:
def nasdaq_processor():
    nasdaq_companes=[]
    stop_words_companies=['corp.','corp','inc','inc.','ltd.','ltd','nasdaq','stock','test','shares','N.A.']
    substrings=['.com', ',']
    holder=[]
    
    # Build list from text file
    with open('nasdaq_companies.txt') as f:
        next(f)
        for line in f:
            company=line.split('|')[1].split('-')[0].strip()
            nasdaq_companes.append(company)
    
    
    # Remove stop words and substrings
    for comp in nasdaq_companes:
        split_companies=comp.split(" ")

        for i in split_companies:

            # Remove .com,',', etc.
            for substring in substrings:
                if substring in i:
                    i=i.replace(substring,'')

            # Dont include stop words   
            if i.lower() not in (string.lower() for string in stop_words_companies):
                holder.append(i)

    # Remove single elements
    for comp in holder:
        if len(comp)<=1:
            try:
                while True:
                    holder.remove(comp)
            except ValueError:
                pass
    
    # Remove duplicates
    holder = list(dict.fromkeys(holder))
    
    return (holder)

In [79]:
def remove_no_content_articles(df):
    
    indexes_to_drop = []   
    # Include only articles that mention companies in the nasdaq list
    for i, row in df.iterrows():
        
        content= df.at[i, 'Content']
        
        if content == False or type(content) != str:
            
            indexes_to_drop.append(i)
    
    df=df.drop(df.index[indexes_to_drop])
    df = df.reset_index(drop=True)

    return df

In [80]:
def remove_no_mention_articles(df, nasdaq_list):
    
    indexes_to_drop = []   
    # Include only articles that mention companies in the nasdaq list
    for i, row in df.iterrows():
        
        content= df.at[i, 'Content']
        
        res = [ele for ele in nasdaq_list if(ele in content)]
        
        if bool(res) == False:
            
            indexes_to_drop.append(i)
    
    df=df.drop(df.index[indexes_to_drop])
    df = df.reset_index(drop=True)
    
    return df

In [81]:
nasdaq_companes=nasdaq_processor()

In [82]:
df=remove_no_content_articles(df)

In [83]:
df=remove_no_mention_articles(df, nasdaq_companes)

In [127]:
# df.to_csv('data_backup/ouput_with_content_with_companies.csv', index=False)
df=pd.read_csv("data_backup/ouput_with_content_with_companies.csv")

In [128]:
df.head()

,Id,Title,DocumentIdentifier,Content,DATE,V2Themes,V2Organizations,AllNames,Extras
0,0,'They proved me wrong.' Investor who passed on...,https://www.geekwire.com/2021/proved-wrong-inv...,Jason Stoffer was still early in his career at...,20210918181500,"{'TAX_FNCACT_HERO': 1, 'MEDIA_MSM': 1, 'ECON_I...",{'Western Union': 1},"{'Remitly Photo': 1, 'Western Union': 1, 'Matt...",<PAGE_LINKS>https://ringingthebell.substack.co...
1,1,TikTok rival Kuaishou becomes the latest Chine...,https://kvia.com/news/2021/11/01/tiktok-rival-...,"By Michelle Toh, CNN Business\n\nKuaishou, the...",20211101073000,"{'GENERAL_GOVERNMENT': 1, 'EPU_POLICY_GOVERNME...",{'Cnn': 1},"{'Michelle Toh': 1, 'Cheng Yixiao': 1, 'Zhang ...",<PAGE_LINKS>https://edition.cnn.com/2021/03/18...
2,2,SoftBank : backed VTEX surges 32% in NYSE debu...,https://www.marketscreener.com/quote/stock/SOF...,SoftBank Group-backed VTEX's stock opened at $...,20210721191500,"{'TAX_FNCACT_COORDINATORS': 1, 'WB_2299_PIPELI...","{'Morgan Stanley': 1, 'Mcdonald Corp': 1, 'Son...","{'SoftBank Group-backed': 1, 'Sony Corp': 1, '...",<PAGE_AUTHORS>MarketScreener</PAGE_AUTHORS><PA...
3,4,Human capital firm Paycor pops in Nasdaq debut...,https://www.reuters.com/article/paycor-ipo/upd...,The Nasdaq logo is displayed at the Nasdaq Mar...,20210721183000,"{'IDEOLOGY': 1, 'GENERAL_HEALTH': 1, 'HEALTH_V...","{'Thomson Reuters Trust Principles': 1, 'Goldm...","{'United States': 2, 'Apax Partners': 1, 'Ohio...",<PAGE_LINKS>http://thomsonreuters.com/en/about...
4,5,"ACADIA Pharmaceuticals Inc. (NASDAQ:ACAD), Bea...",https://www.benzinga.com/general/biotech/21/07...,The following post was written and/or publishe...,20210721163000,"{'TAX_DISEASE_PARKINSON_DISEASE': 1, 'TAX_DISE...","{'Amgen': 1, 'Acadia Pharmaceuticals': 1, 'Dru...","{'Nasdaq Biotech Index': 2, 'Eli Lilly': 2, 'D...",<PAGE_LINKS>https://pixabay.com/;https://pixab...


# Part 2.3 Document Lengths

In [129]:
def doc_lengths(df):
    
    df["Content_length"] = 'NaN'
    
    # Include only articles that mention companies in the nasdaq list
    for i, row in df.iterrows(): 
        article_id=df.at[i, 'Id']
        content= df.at[i, 'Content']
        len_content=len(content)
        
        df.at[i, 'Content_length'] = len_content
        
    return df

In [130]:
df=doc_lengths(df)

In [145]:
df.head()

,Id,Title,DocumentIdentifier,Content,DATE,V2Themes,V2Organizations,AllNames,Extras,Content_length
0,0,'They proved me wrong.' Investor who passed on...,https://www.geekwire.com/2021/proved-wrong-inv...,Jason Stoffer was still early in his career at...,20210918181500,"{'TAX_FNCACT_HERO': 1, 'MEDIA_MSM': 1, 'ECON_I...",{'Western Union': 1},"{'Remitly Photo': 1, 'Western Union': 1, 'Matt...",<PAGE_LINKS>https://ringingthebell.substack.co...,1910
1,1,TikTok rival Kuaishou becomes the latest Chine...,https://kvia.com/news/2021/11/01/tiktok-rival-...,"By Michelle Toh, CNN Business\n\nKuaishou, the...",20211101073000,"{'GENERAL_GOVERNMENT': 1, 'EPU_POLICY_GOVERNME...",{'Cnn': 1},"{'Michelle Toh': 1, 'Cheng Yixiao': 1, 'Zhang ...",<PAGE_LINKS>https://edition.cnn.com/2021/03/18...,2196
2,2,SoftBank : backed VTEX surges 32% in NYSE debu...,https://www.marketscreener.com/quote/stock/SOF...,SoftBank Group-backed VTEX's stock opened at $...,20210721191500,"{'TAX_FNCACT_COORDINATORS': 1, 'WB_2299_PIPELI...","{'Morgan Stanley': 1, 'Mcdonald Corp': 1, 'Son...","{'SoftBank Group-backed': 1, 'Sony Corp': 1, '...",<PAGE_AUTHORS>MarketScreener</PAGE_AUTHORS><PA...,1882
3,4,Human capital firm Paycor pops in Nasdaq debut...,https://www.reuters.com/article/paycor-ipo/upd...,The Nasdaq logo is displayed at the Nasdaq Mar...,20210721183000,"{'IDEOLOGY': 1, 'GENERAL_HEALTH': 1, 'HEALTH_V...","{'Thomson Reuters Trust Principles': 1, 'Goldm...","{'United States': 2, 'Apax Partners': 1, 'Ohio...",<PAGE_LINKS>http://thomsonreuters.com/en/about...,1991
4,5,"ACADIA Pharmaceuticals Inc. (NASDAQ:ACAD), Bea...",https://www.benzinga.com/general/biotech/21/07...,The following post was written and/or publishe...,20210721163000,"{'TAX_DISEASE_PARKINSON_DISEASE': 1, 'TAX_DISE...","{'Amgen': 1, 'Acadia Pharmaceuticals': 1, 'Dru...","{'Nasdaq Biotech Index': 2, 'Eli Lilly': 2, 'D...",<PAGE_LINKS>https://pixabay.com/;https://pixab...,11695


In [132]:
def content_length_dict(df):
    content_len_d={}
    for i, row in df.iterrows(): 
        article_id=int(df.at[i, 'Id'])
        content_len= df.at[i, 'Content_length']
        content_len_d[article_id]=content_len
    
    return content_len_d

In [133]:
content_length_d=content_length_dict(df)

In [134]:
# Output document lengths
with open('content_length.json', 'w') as fp:
    json.dump(content_length_d, fp)

# Part 3 - Write to SQL

In [ ]:
# For each row, take id and write it to the SQL database

# Part 4 - Create Inverted Index

In [139]:
def processor(string):
    
    # Convert to lower case
    string=string.lower()
    
    # Tokenize
    pattern = r"\w+"
    tokens=re.findall(pattern, string)

    # Stop word removal
    with open('stop_words.txt') as f:
        stop_words = [line.rstrip() for line in f]
    
    filtered_phrase = []

    for w in tokens:
        if w not in stop_words:
            filtered_phrase.append(w)

    tokens=filtered_phrase.copy()
    
    # Stemmer
    ps = PorterStemmer()
    tokens = [ps.stem(word) for word in tokens]
    
    return tokens

In [148]:
def create_inverted_index(df):
    
    # Create dictionary {word: {doc_num: [positions]}}, {word: {doc_num: [positions]}}
    d={}
    # Create dictionary for frequencies {word:count, word:count}
    d_frequencies={}

    for i, row in df.iterrows():

        content=df.loc[df.index[i], 'Content']
        title=df.loc[df.index[i], 'Title']
        
        # If no title exists
        if type(title) != str:
            combined=content
        
        else:
            # Headline and text combination
            combined=content+title

        # Tokenization process
        tokens=processor(combined)

        # Get doc number
        doc_id=int(df.at[i, 'Id'])


     # For each token (word) check if it exists in inverted index. Also create word index variable.
        word_index = 0 

        for word in tokens:
            word_index +=1

            # Check word exists in d={word: {doc_num: [positions], doc_num: [positions]}, word: {doc_num, [positions]}]
            if word in d:
                # Get values = {doc_num: [positions], doc_num: [positions]}
                values=d[word]

                # if doc_num already exists - i.e. that same word appeared earlier in the sentence
                if doc_id in values:
                    positions=values[doc_id]
                    positions.append(word_index)

                    d[word][doc_id] = positions


                # If value's doc_num doesn't exist, add one
                else:
                    d[word][doc_id]=[word_index]

            else:
                d[word]= {doc_id: [word_index]}
    
    return d

In [149]:
d=create_inverted_index(df)

In [152]:
# Output document lengths
with open('inverted_index.json', 'w') as fp:
    json.dump(d, fp)

In [ ]:
d['alphabet']

In [ ]:
df.loc[df['Id'] == 913, 'DocumentIdentifier']

# Ghadi

In [2]:
import os
import pandas as pd
import json 

In [12]:
# Opening JSON file
with open('inverted_index.json') as json_file:
    inverted_index = json.load(json_file)

In [13]:
# Opening JSON file
with open('content_length.json') as json_file:
    lengths = json.load(json_file)

In [14]:
df=pd.read_csv("ouput_with_content_with_companies.csv")

### # ----- Testing ------

In [31]:
# Identify doc 913 for example
inverted_index['meta']

{'372': [487],
 '457': [403],
 '466': [26, 121],
 '789': [14],
 '1585': [46],
 '1668': [188],
 '1672': [188],
 '1712': [388],
 '1795': [289],
 '1800': [25, 76, 129],
 '1926': [5, 97, 119, 218, 247, 258],
 '1927': [5, 97, 119, 218, 247, 258],
 '1969': [66, 91],
 '1978': [159, 203],
 '2018': [59],
 '2120': [197, 216, 432],
 '2143': [16, 48, 198, 395, 453, 496],
 '2537': [110],
 '2619': [51, 131, 138, 151, 258],
 '2828': [12],
 '2979': [201, 1134],
 '2989': [80]}

In [33]:
# Print doc with doc id = 913
list(df.loc[df['Id'] == 2619, 'DocumentIdentifier'])

['https://www.fool.co.uk/2022/01/22/three-metaverse-stocks-im-buying-right-now/']